In [11]:
import numpy as np

#每个老虎机的中奖概率,0-1之间的均匀分布
probs = np.random.uniform(size=10)

#记录每个老虎机的返回值
rewards = [[1] for _ in range(10)]

probs, rewards

(array([0.48901417, 0.04928586, 0.11198927, 0.43497977, 0.4112062 ,
        0.26058366, 0.86182331, 0.26397791, 0.12256801, 0.95078981]),
 [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]])

In [12]:
import random


#随机选择的概率递减的贪婪算法
def choose_one():
    #求出每个老虎机各玩了多少次
    played_count = [len(i) for i in rewards]
    played_count = np.array(played_count)

    #求出上置信界
    #分子是总共玩了多少次,取根号后让他的增长速度变慢
    #分母是每台老虎机玩的次数,乘以2让他的增长速度变快
    #随着玩的次数增加,分母会很快超过分子的增长速度,导致分数越来越小
    #具体到每一台老虎机,则是玩的次数越多,分数就越小,也就是ucb的加权越小
    #所以ucb衡量了每一台老虎机的不确定性,不确定性越大,探索的价值越大
    fenzi = played_count.sum()**0.5
    fenmu = played_count * 2
    ucb = fenzi / fenmu
    # print(f"{fenzi=}\n{fenmu=}\n{ucb=}")

    #ucb本身取根号
    #大于1的数会被缩小,小于1的数会被放大,这样保持ucb恒定在一定的数值范围内
    ucb = ucb**0.5

    #计算每个老虎机的奖励平均
    rewards_mean = [np.mean(i) for i in rewards]
    rewards_mean = np.array(rewards_mean)

    #ucb和期望求和
    ucb += rewards_mean

    return ucb.argmax()


choose_one()

0

In [13]:
def try_and_play():
    i = choose_one()

    #玩老虎机,得到结果
    reward = 0
    if random.random() < probs[i]:
        reward = 1

    #记录玩的结果
    rewards[i].append(reward)


try_and_play()

rewards

[[1, 0], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

In [15]:
def get_result():
    # reset rewards
    global rewards
    rewards = [[1] for _ in range(10)]

    #玩N次
    for _ in range(5000):
        try_and_play()

    #期望的最好结果
    target = probs.max() * 5000

    #实际玩出的结果
    result = sum([sum(i) for i in rewards]) - len(rewards)

    return target, result, "{:.3f}%".format(result / target * 100)


get_result()

(4753.949060674144, 4342, '91.335%')